In [1]:
import pandas as pd
import numpy as np
import os
import shutil

In [ ]:
df_interactions = pd.read_csv('../data/userid_trackid_timestamp_m4a.tsv', sep='\t')
print(df_interactions.shape)
df_interactions.head()

(1000000, 3)


,user_id,track_id,timestamp
0,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:42:38
1,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:38:53
2,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:35:08
3,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:31:23
4,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:27:38


In [ ]:
df_interactions['period'] = pd.to_datetime(df_interactions['timestamp']).dt.to_period('Y')
df_interactions = df_interactions[df_interactions['period'] == '2018']
print(df_interactions.shape)
df_interactions.head()

(316436, 4)


,user_id,track_id,timestamp,period
0,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:42:38,2013
1,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:38:53,2013
2,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:35:08,2013
3,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:31:23,2013
4,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:27:38,2013


In [15]:
# binarization: we convert the interactions to binary implicit feedback with a threshold of 2 on the interaction counts (reducing false-positive interactions)
def filter_interactions(listening_history: pd.DataFrame, min_interactions: int = 2, verbose: bool = True):
    lhs_count = listening_history.value_counts(subset=['user_id', 'track_id'])
    lhs_count = lhs_count[lhs_count >= min_interactions]
    listening_history = listening_history.set_index(['user_id', 'track_id']).loc[lhs_count.index]
    listening_history = listening_history.reset_index()
    return listening_history

df_interactions = filter_interactions(df_interactions, min_interactions=2)
print(df_interactions.shape)
df_interactions.head()

(249403, 4)


,user_id,track_id,timestamp,period
0,24983,QJti5s20kD0pxWj0,2013-04-21 03:53:46,2013
1,24983,QJti5s20kD0pxWj0,2013-04-21 03:43:37,2013
2,24983,QJti5s20kD0pxWj0,2013-04-21 03:33:25,2013
3,24983,QJti5s20kD0pxWj0,2013-04-21 03:23:11,2013
4,24983,QJti5s20kD0pxWj0,2013-04-09 02:11:49,2013


In [6]:
df_interactions

,user_id,track_id
0,0,07qIMK4LPj4gT40H
2,0,0SF1CZU2vmCyF9Uw
6,0,0oyRcxsZqlruwJmE
8,0,10iGPqXD7nmgAxZ2
9,0,13LIhGPoNkTFDIuK
...,...,...
5349482,120321,zVaurSbD85VqTSdD
5349483,120321,zWwVtC0NZ2QoqAf8
5349485,120321,zi7vUgfKC012duwE
5349487,120321,zojyhkGcRUgoHi88


In [23]:
# sparsity of dataset
n_users = df_interactions['user_id'].nunique()
n_items = df_interactions['track_id'].nunique()
n_interactions = df_interactions.shape[0]
sparsity = 1 - n_interactions / (n_users * n_items)
sparsity

0.9864761642161429

In [24]:
n_items

242

In [25]:
n_users

4436

In [ ]:
# export
df_interactions.to_csv('../data/userid_trackid_timestamp_onion2018.tsv', sep='\t', index=False)